# Project 1 - Air Resistance in a 2D Projectile

Start by importing `numpy` for numerical computations and `matplotlib` for plotting. That should be all we need for now.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

Now we need to model our system in terms of forces acting on the projectile. Gravity is the simplest force:

$$ F_g = -mg $$

In [ ]:
def gravitational_force(m, g=9.80665, dimensions=2, vertical_axis=1):
    ''' Return gravitational force on a projectile subject to gravity.
        
        Returns downward force vector in netwons. 

                  F_g = g * m  

        Args:
            
            m (np.array):           mass of the object (kg) 
            g (np.float64):         gravitational acceleration (m/s**2)
            dimensions (int):       number of dimensions in return vector
            vertical_axis (int):    axis which gravitional force acts on

        Returns:

            (np.array):             gravitational force vector (N)
    '''

    force = np.zeros(dimensions)
    force[vertical_axis] = -g * m
    
    return force

Drag force is a little more complex, it involves calculation of air resistance.

$$
    F_d = \frac{D \rho A}{2} |v|^2
$$

We will need to be able to calculated the magnitude of a multi-dimensional velocity vector, so we should create a quite helper function.

In [ ]:
def magnitude(x):
    ''' Return mangitude of vector x. 

        Args:

            x (np.array):   input vector

        Returns
            
            (np.float64):   magnitude of x 
    '''

    return np.sqrt(np.sum(x * x))

The drag force is orientated in the opposite direction of the velocity vector. The easiest way to handle this geometrically is using unit vectors, hence another helper function.

In [ ]:
def unit_vector(x):
    ''' Return unit vector for vector x. 

        Args:

            x (np.array):   input vector

        Returns
            
            (np.array):     unit vector for x
    '''

    return x / magnitude(x)

Now we can use these two tools to finally build our drag force function.

In [ ]:
def drag_force(v, D, rho, A):
    ''' Return drag force on a projectile subject to air resistance.
        
        Returns a force vector in the opposite direction as the velocity
        vector with a magnitude determined by the drag equation below.

                    F_d = (D * rho * A) / 2 * v**2

        Args:
            
            v (np.array):       velocity vector 
            D (np.float64):     drag coefficient
            rho (np.float64):   air density (units)
            A (np.float64):     cross-sectional area (m**2)

        Returns:

            (np.array):         drag force vector 
    '''

    F_d = ((D * rho * A) / 2) * magnitude(v)**2
    
    return F_d * (-1) * unit_vector(v)

Acceleration of the projectile follows Newtons Law:

$$
    \sum F_i = ma
$$

We have already accounted for our major forces (i.e. gravity and drag), so now we just build them into a function.

In [ ]:
def acceleration(x, v, D, rho, A, m=1):
    ''' Return the acceleration vector for a projectile subject to 
        air resistance.
        
        Uses:

                        a = F_net / m

        Args:
            
            v (np.array):       velocity vector 
            D (np.float64):     drag coefficient
            rho (np.float64):   air density (units)
            A (np.float64):     cross-sectional area (m**2)
            m (np.float64):     mass of the projectile (kg)

        Returns:

            (np.array):         acceleration vector 
    '''

    F_d = drag_force(v, D, rho, A)
    F_g = gravitational_force(m)

    return (F_d + F_g) / m

Finally, we copy over our Euler Method from GA3. We spent the time then to ensure that the function could handle multi-dimensional input. This latest version is built on super-fast numpy arrays. Numpy is a python wrapper for very fast, very close to the metal C functions. This *will* run faster than MATLAB.

In [ ]:
def euler_method(acceleration, t0, tf, dt, x0=0, v0=0, modified=False, **kwargs):
    ''' Returns t, x, v, and a arrays for Euler Method

    Args:
        acceleration (func):    acceleration function **

        t0 (float):             initial time (s)
        tf (float):             final time (s)
        dt (float):             time step (s)
        
        x0 (np.array):          initial displacement (m)
        v0 (np.array):          initial velocity (m/s)
        **kwargs:               any additional parameters required for
                                    acceleration function.

    Returns:
        np.float64 (np.array):  list of times (s)
        np.float64 (np.array):  list positions (m)
        np.float64 (np.array):  list velocities (m/s)
        np.float64 (np.array):  list accelerations (m/s**2)
    '''

    t = np.arange(t0, tf, dt, dtype=np.float64)

    points = t.size
    dimensions, = x0.shape

    x = np.empty([points, dimesnions])
    v = np.empty([points, dimesnions])
    a = np.empty([points, dimesnions])

    x[0] = x0
    v[0] = v0
    a[0] = acceleration(x0, v0, **kwargs)

    for i in range(1, points):
        a[i] = acceleration(x[i-1], v[i-1], **kwargs)

        if modified:
            v[i] = v[i-1] + dt * a[i]
            x[i] = x[i-1] + dt * v[i]
        else:
            x[i] = x[i-1] + dt * v[i-1]
            v[i] = x[i-1] + dt * a[i-1]

    return t, x, v, a

We just need one last helper function. The initial velocity of a projectile is often specified as a velocity magnitude and an incline. The most convient form for our calculations is x, y (, and z) velocity components. They are found through some basic trigonometry.

$$ x = v_0 \cos(\phi) $$
$$ y = v_0 \sin(\phi) $$

To make thinigs unnecessarily complex, this function also handles azimuth. I am considering doing these problems in 3-dimensions so I can play with coriolis, ship velocity and bearing, and wind velocity. 

In [ ]:
def launch_vector(mag, incline, azimuth=False, radians=False):
    ''' Return v0 vector for given scalar magnitude and incline above the horizon. 

        Args:
            magnitude (float):      magnitude of v0 (m/s)
            incline (float):        angle above horizon (degrees)
            azimuth (float):        angle in plane of horizon (degrees)
            radians (bool):         option for using radian incline

        Returns:
            np.array (np.float64):  vector with launch velocity components
    '''
    
    if not radians:
        incline = incline * (np.pi / 180.0)

        if azimuth:
            azimuth = azimuth * (np.pi / 180.0)
    
    if azimuth:
        z = mag * np.sin(incline)
        h = mag * np.cos(incline)

        x = h * np.cos(azimuth)
        y = h * np.sin(azimuth)

        return np.asarray([x, y, z])

    else:
        x = mag * np.cos(incline)
        y = mag * np.sin(incline)

        return np.asarray([x, y])


## Exercise 1
### 2D Projectile with Air Resistance
#### Requirements

1. Build a computational model of a 2D projectile subject to air resistance.
2. Assume air density and the drag coefficient remain constant.
3. Model inputs must be magnitude and angle of initial velocity.
4. Determine accuracy of model.
5. Determine adequate $\Delta t$ for model.
6. Plot a set of parameters of your choosing.

## Exercise 2
### Shape of the 2D Projectile Trajectory
#### Requirements

1. Compare your model to a parabola, are they the same?

# Exercise 3
### Angle for Maximum Projectile Range
#### Requirements

1. What angles of $\theta$ ($\pm 0.1^\circ$) maximize the range? Use three very differnt drag coefficients.
2. Plot all three trajectories together. Label max range, angle, and drag coefficient for each series.


# Exercise 4
### 2D Model with Air Resistance--Naval Gun
#### Requirements

1. Goto http://www.navweaps.com/Weapons/WNUS_1650_mk6.php and retrieve real parameters (i.e. mass, frontal area, initial velocity) for a 16" naval gun.
2. Account for changes in air density.
3. Determine a constant drag coefficient, D, by fiting your model to the following data. Use "new gun" muzzle velocities.


|**Elevation ($^\circ$)**|**AP Mark 8 (m)**|**HC Mark13 (m)**| 
|:-----:|:-----:|:-----:|
|10|16139|16642| 
|15|21854|22037| 
|20|26518|26335| 
|25|30450|29901| 
|30|33558|32918| 
|35|36119|35342| 
|40|37884|37163| 
|45|38720|38059|     

# Exercise 5
### Angle for Maximum Range
#### Requirements

1. Find the maximum angle for the naval projectile using the accurate paramter from Exercise 4.
2. Is the angle greater or smaller than $45^\circ$